# MTL732 
# Ravi Pushkar (2018MT60790) || Satendra Singh Parasher (2018MT60793)

## Download and Install Pinguin

In [89]:
pip install pingouin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importing Libraries

In [243]:
import os
import csv
import glob
import numpy as np
import pandas as pd
import cvxpy as cp
import pingouin as pg
from scipy.stats import norm
from numpy.linalg import matrix_power
# from numpy.linalg import norm
from scipy.linalg import fractional_matrix_power
from pingouin import multivariate_normality

## Mounting the drive and getting the data set


In [244]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [242]:
path1 = "/content/sample_data/MTL732_IND"
csv_files = glob.glob(os.path.join(path1, "*.csv"))

## Calculating the Mean AND variance covariance, and checking if the data is multivariate


In [257]:
class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 
  

dictionary = my_dictionary() 
count = 20
for f in csv_files:

    if(count == 0):
      break
    file_name = os.path.basename(f)
    key = os.path.splitext(file_name)[0]
    df = pd.read_csv(f)
    dictionary.add(key, (df["Close"]-df["Open"])/df["Open"])
    count -= 1
    
data = pd.DataFrame.from_dict(dictionary)
print(multivariate_normality(data , alpha = 0.1))

df1 = data.dropna()

Mean = []

security_count = 0
for col in df1:
  security_count += 1
  Mean.append(df1[col].mean())

mean_vector = np.array(Mean)
mean_vector = mean_vector.reshape((security_count, 1))
var_cov = df1.cov()

# Mean = df1.mean()
# print(Mean.shape)
# print(Mean)
# print(var_cov)

mean_vector.shape, var_cov.shape

HZResults(hz=1.1319124916377437, pval=0.0, normal=False)


((20, 1), (20, 20))

## Optimisation Problem


In [258]:
x = cp.Variable((security_count,1))

In [259]:
alfa = 0.90
ninv_alpha = norm.ppf(alfa)
ninv_alpha

1.2815515655446004

In [261]:
cov_var_power_half = fractional_matrix_power(var_cov,1/2)
# cov_var_power_half
cov_var_power_half.shape
assert(np.linalg.norm(cov_var_power_half @ cov_var_power_half - var_cov)< 1e-8)

# cv_power_half
# cov_var_power_half.shape

In [262]:
d = 0.019
constraint_1 = ninv_alpha * cp.norm(cov_var_power_half @ x) - mean_vector.T @ x<=d
constraint_2 = x >= 0
constraint_3 = cp.sum(x) == 1


In [263]:
constraints = [constraint_1, constraint_2, constraint_3]
obj=cp.Maximize(mean_vector.T @ x)
problem = cp.Problem(obj, constraints)
problem.solve()
print(problem.status)
print(x.value.T)

optimal
[[4.48078001e-11 3.30744372e-11 4.06553572e-11 5.74718625e-11
  2.94255094e-11 4.82028658e-11 5.02243538e-11 2.41435934e-11
  5.21451607e-11 8.54601422e-11 3.63053350e-11 9.99999999e-01
  2.16695289e-11 4.87314795e-11 5.15732710e-11 2.11237499e-11
  4.51815549e-11 3.91116610e-11 4.57573672e-11 5.59312764e-11]]


In [268]:
def find_optimal(mu, sigma, alpha, d):
  count = len(mu)
  w = cp.Variable(count)
  ninv_alpha = norm.ppf(alpha)
  # ninv_alpha
  constraint_1 = ninv_alpha * cp.norm(sigma @ w) - mu.T @ w <= d
  constraint_2 = w >= 0
  constraint_3 = cp.sum(w) == 1
  constraints = [constraint_1, constraint_2, constraint_3]
  obj=cp.Maximize(mean_vector.T @ w)
  problem = cp.Problem(obj, constraints)
  problem.solve()
  return w.value.T, w.value.T @ mu

In [269]:
find_optimal(mean_vector, cov_var_power_half, alfa, d)

(array([4.48078001e-11, 3.30744372e-11, 4.06553572e-11, 5.74718625e-11,
        2.94255094e-11, 4.82028658e-11, 5.02243538e-11, 2.41435934e-11,
        5.21451607e-11, 8.54601422e-11, 3.63053350e-11, 9.99999999e-01,
        2.16695289e-11, 4.87314795e-11, 5.15732710e-11, 2.11237499e-11,
        4.51815549e-11, 3.91116610e-11, 4.57573672e-11, 5.59312764e-11]),
 array([0.00030895]))